In [1]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import numpy as np
import os
from keras.regularizers import l2,l1

Using TensorFlow backend.


In [2]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('acc')>0.998):
            print("\n Reached 99.8% accuracy. So cancelling training")
            self.model.stop_training=True

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()  

In [4]:
X_train=X_train.reshape(60000, 28, 28, 1)
X_test=X_test.reshape(10000, 28, 28, 1)
X_train = X_train / 255                      #Normalization - greyscale images contain 255 pixels(speeds up the process)
X_test = X_test / 255

In [5]:
callback = myCallback()

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),input_shape=(28,28,1),activation="relu",kernel_regularizer=l2(0.01),activity_regularizer=l1(0.01)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),                                              #Sequential layers
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")])

W0821 22:31:32.264965 11120 deprecation.py:506] From C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2704)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                173120    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 173,930
Trainable params: 173,930
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer = "adam", loss="sparse_categorical_crossentropy", metrics = ["accuracy"])

In [8]:
model.fit(X_train, y_train, epochs = 20, validation_split=0.1, callbacks = [callback])

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 21s 397us/sample - loss: 0.5543 - acc: 0.9346 - val_loss: 0.2312 - val_acc: 0.9733
Epoch 2/20
54000/54000 [==============================] - 21s 395us/sample - loss: 0.2144 - acc: 0.9697 - val_loss: 0.1760 - val_acc: 0.9778
Epoch 3/20
54000/54000 [==============================] - 22s 400us/sample - loss: 0.1672 - acc: 0.9755 - val_loss: 0.1496 - val_acc: 0.9797
Epoch 4/20
54000/54000 [==============================] - 21s 391us/sample - loss: 0.1420 - acc: 0.9788 - val_loss: 0.1338 - val_acc: 0.9812
Epoch 5/20
54000/54000 [==============================] - 24s 439us/sample - loss: 0.1261 - acc: 0.9811 - val_loss: 0.1422 - val_acc: 0.9777
Epoch 6/20
54000/54000 [==============================] - 21s 385us/sample - loss: 0.1144 - acc: 0.9835 - val_loss: 0.1177 - val_acc: 0.9795
Epoch 7/20
54000/54000 [==============================] - 20s 378us/sample - loss: 0.1052 - acc: 0.9847 -

In [12]:
model.evaluate(x = X_test, y = y_test , verbose = 1)

10000/10000 [==============================] - 1s 116us/sample - loss: 0.1078 - acc: 0.9774


[0.10777754781842232, 0.9774]

In [13]:
model.metrics_names

['loss', 'acc']

In [15]:
img = Image.open('C:\\Users\\user\\Documents\\ML\\MLProjects\\digits/5.png').convert("L")
img = img.resize((28,28))
im2arr = np.array(img)
#print(im2arr)
im2arr = im2arr.reshape(1,28,28,1)
y_pred = model.predict_classes(im2arr)
print(y_pred)

[3]


In [14]:
import struct

import numpy as np

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)